In [2]:
from scipy import io
import numpy as np
import pandas as pd

from classes.matlab_reader import matlab_reader
from classes.grid_funcs import map_grid,dataset,data_wizard

matlab = matlab_reader('AIRS_125KM_2018/20180101_AIRS_3DST-1_125km_grid')
# matlab = matlab_reader('AIRS_40KM_2022/20220101_AIRS_3DST-1_40km_grid.mat')

grid = map_grid(grid_size=125)
wizard = data_wizard()

# "BG" : 0,
# "k" : 1,
# "bg" : 2,
# "l" : 3,
# "m" : 4,
# "mfx" : 5,
# "mfy" : 6,


temp_data = matlab.select(
    hemisphere= 'nh',
    data_field='mfy',
    daytime='night',
    altitude=0)   

In [4]:
temp_data = matlab.select(
    hemisphere= 'nh',
    data_field='k',
    daytime='night',
    altitude=2)   
    
select_area_indexer = grid.select_area_indexer(
    min_lat= 60,
    max_lat= 90,
    min_lon= -180,
    max_lon= 180)

data = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = temp_data)

In [5]:
print(data)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
